<center><img src="./images/Header.png" 
    Width=2000">|

## 1. Business Understanding

### 1.1. Business Objective 
This project aims to determine which movies are currently doing the best at the box office. Several sources of data are used (see [here](https://github.com/setare92-ha/project_eda_movie_profit_analysis/tree/main/zippedData)). These sources include data from [Box Office Mojo](https://www.boxofficemojo.com/), [IMDB](https://www.imdb.com/?ref_=nv_home), [Rotton Tomatoes](https://www.rottentomatoes.com/), [The Movie DataBase](https://www.themoviedb.org/?language=en-US)(TMDB) and [TheNumbers](https://www.the-numbers.com/). The study and its findings will serve stakeholders trying to start a new venture in the movie industry, such as creating a movie studio, or streaming platforms trying to determine movies that would be profitable additions to their collection. 

### 1.2. Background Information
Different metrics may be used to determine the success or profitability of a movie. We may come to different conclusions based on the choice of metric (see [this article](https://www.hollywoodreporter.com/movies/movie-news/what-is-profitable-movie-ever-1269879/) for reference). Here, we choose Return On Investment (ROI) as an appropriate metric to use. ROI may be defined as $$ROI=\frac{Revenue-Expense}{Expense}\times 100$$. We focus on this fiscal metric to determine the most profitable movies. Later, we will focus on the different aspects of a movie (genre, movie ratings, director, etc.) that may correlate to the movie's success in general.  

### 1.3. Data Mining Goals
We would aim for cleaning the data and retaining as much relevant information as possible. Since we're taking data from various sources, we may need to unify them for a more comprehensive analysis. Each source of data may use different jargon/metrics. It's essential to understand what these metrics represent before we attempt to unify data across sources. 

### 1.4. Project Plan
We will first skim the data sources available and get an understanding of the attributes. Then we will clean the data and prepare it for analysis. The important attributes to keep for the different stages of analysis will be identified. Next, we will draw insights from the data to answer the business question. To make the findings more accessible to the stakeholders, we will summarize them as business recommendations at the end. As is known, any study is as good as its limitations. Therefore, at the very end, we will list the assumptions and limitations of the study and how we could potentially better the study in the future. 

## 2. Data Understanding
In what follows, we take a preliminary look at the data and understand what information we have available.  

In [640]:
import ast
from bs4 import BeautifulSoup
import numpy
import pandas as pd
import requests
import sqlite3
import matplotlib.pyplot as plt

%matplotlib inline

### 2.1. Data from the Box Office Mojo

In [641]:
# import data from Box Office Mojo
df_bom = pd.read_csv("./unzipped_data/bom.movie_gross.csv")
print("summary of the data from Box Office Mojo:")
print(df_bom.info())
df_bom.head(10)

summary of the data from Box Office Mojo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
5,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010
6,Iron Man 2,Par.,312400000.0,311500000,2010
7,Tangled,BV,200800000.0,391000000,2010
8,Despicable Me,Uni.,251500000.0,291600000,2010
9,How to Train Your Dragon,P/DW,217600000.0,277300000,2010


In [642]:
# what percentage of data is missing from the domestic gross & foreign gross columns?
print(
    "% missing data from the domestic gross column:",
    (1 - len(df_bom.dropna(subset=["domestic_gross"])) / len(df_bom)) * 100,
)
print(
    "% missing data from the foreign gross column:",
    (1 - len(df_bom.dropna(subset=["foreign_gross"])) / len(df_bom)) * 100,
)
print(
    "% missing data from the foreign & domestic gross columns combined:",
    (1 - len(df_bom.dropna(subset=["domestic_gross", "foreign_gross"])) / len(df_bom))
    * 100,
)

% missing data from the domestic gross column: 0.8266902863891312
% missing data from the foreign gross column: 39.85828166519043
% missing data from the foreign & domestic gross columns combined: 40.68497195157958


In [643]:
df_bom["foreign_gross"] = (
    df_bom["foreign_gross"].str.replace(",", "", regex=False).astype(float)
)  # convert the foreign gross data type to float
df_bom.dropna(subset=["domestic_gross", "foreign_gross"])[
    ["domestic_gross", "foreign_gross"]
].describe()

,domestic_gross,foreign_gross
count,2.009000e+03,2.009000e+03
mean,4.697311e+07,7.571822e+07
std,8.159966e+07,1.381296e+08
min,4.000000e+02,6.000000e+02
25%,6.650000e+05,3.900000e+06
50%,1.650000e+07,1.930000e+07
75%,5.600000e+07,7.590000e+07
max,9.367000e+08,9.605000e+08


**Observations:**
1. The data from Box Office Mojo has 3387 entries and it contains information on the movie name, studio, and the year it was produced. There's also information on the **domestic gross** and **foregin gross** revenue of the movies. The former is the total revenue a movie earns in its home country (in this case, United States and Canada) while the latter refers to the total revenue a movie earns outside of its home country. This information combined with data on the cost/budget for each movie (missing from the current table) will come in handy for determining the **Return on Investment** metric. One possibility is to use the movie name to read movie_id from the movie_basics table in the IMDB database (loaded in the next section), then scrape the IMDB website to find the budget and add it to this table. We will attempt to do this in the Data Preparation section. 
2. The data type for foreign gross is object (text). We need to change it to float. 
3. We also notice that several entries are missing from the domestic gross (less than 1%) and foreign gross (~40%) columns. This data will need cleaning (including the removal of NaNs) before it can be used in our analysis. 

### 2.2. Data from IMDB

In [644]:
# import data from IMDB
conn = sqlite3.connect("./unzipped_data/im.db")  # open up a connection
cursor = conn.cursor()
# table name query
table_name_query = (
    """SELECT name AS table_name FROM sqlite_master WHERE type='table';"""
)
imdb_sql_table_names = pd.read_sql(table_name_query, conn)
imdb_sql_table_names.head(10)

,table_name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


Let's explore each table and understand what useful information they might contain. Below we define a function that queries each table to show all its columns. We will reuse this function to take a peek at the data in all tables.

In [645]:
def show_table_contents(table_name):
    read_table_query = f"""SELECT *
                        FROM {table_name}"""
    print(f"This is the content of {table_name}")
    print(pd.read_sql(read_table_query, conn).head());

#### 1.2.1. IMDB Data: Table Contents

In [646]:
[
    show_table_contents(table_name)
    for table_name in list(imdb_sql_table_names["table_name"])
];

This is the content of movie_basics
    movie_id                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2  tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3  tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4  tt0100275         The Wandering Soap Opera       La Telenovela Errante   

   start_year  runtime_minutes                genres  
0        2013            175.0    Action,Crime,Drama  
1        2019            114.0       Biography,Drama  
2        2018            122.0                 Drama  
3        2018              NaN          Comedy,Drama  
4        2017             80.0  Comedy,Drama,Fantasy  
This is the content of directors
    movie_id  person_id
0  tt0285252  nm0899854
1  tt0462036  nm1940585
2  tt0835418  nm0151540
3  tt0835418  nm0151540
4  tt0878654  nm00

**Observations:** }This database has some useful information about **movie ratings** and **number of votes** as well as **directors** and **actors**. Since we would like to focus on a fiscal measure such as ROI, this data does not seem particularly necessary to include in our analysis. However, we may use this data at the end to study correlations between high ratings and other factors mentioned above. 

### 2.3. Data from Rotten Tomatoes

In [647]:
# import data on movie reviews from rotton tomatoes
df_rt_movie_info = pd.read_csv("./unzipped_data/rt.movie_info.tsv", delimiter="\t")
print("Summary of the data on movie info from Rotton Tomatoes:")
print(df_rt_movie_info.info())
df_rt_movie_info.head()

Summary of the data on movie info from Rotton Tomatoes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB
None


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


The data from Rotten Tomatoes contains information on the **total revenue generated by a movie** (box_office). It also contains other interesting information such as **genre** and **rating**. What is striking though is that the box_office field has a high percentage of missing data (roughly 78%). It's not clear how useful this data will be in terms of considering the revenue. The other problem is that the name of the movies is not listed so there is no way to cross-reference this table with other tables. 

In [648]:
df_rt_reviews = pd.read_csv("./unzipped_data/rt.reviews.tsv", sep="\t", encoding="mbcs")
print(df_rt_reviews.info())
df_rt_reviews.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB
None


,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


This table lists the full reviews people left for movies. The data regarding movie ratings can be combined with that from IMDB if we do an analysis that looks at whether higher ratings correlate with better fiscal outcomes. Further down the line, it may also be possible to train a model that can classify the reviews as positive or negative. Then use that to do further studies on how different movies were generally received. We abandon this idea for now. 

### 2.4. Data from TheMovieDB

In [649]:
# import data from the movie db
df_movie_tmdb = pd.read_csv("./unzipped_data/tmdb.movies.csv", index_col=0)
print(df_movie_tmdb.info())
df_movie_tmdb.head()

<class 'pandas.core.frame.DataFrame'>
Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB
None


,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


This table has information on movie ratings (**vote_average**) and **popularity**. The popularity column does not have a specific range and is determined based on several factors including the recentness of the release and the number of votes. This table also includes information on the **genres**. However, they're listed as ids. If we intend to use this information, we will need to first map the ids to the actual genres using a mapping that TMDB provides itself. I created a separate table that does this using the information [here](https://www.themoviedb.org/talk/5daf6eb0ae36680011d7e6ee). 

In [650]:
df_genre_tmdb = pd.read_csv("./unzipped_data/tmdb.genres.csv")
df_genre_tmdb.head()

,genre,id
0,Action,28
1,Adventure,12
2,Animation,16
3,Comedy,35
4,Crime,80


Once we integrate the genres with this table, we can join this table with the tables from IMDB and Rotton Tomatoes containing similar information. We shelf these ideas for the last stage of the analysis.

### 2.5. Data from TheNumbers

In [651]:
# import data from The Numbers
df_thenumbers = pd.read_csv("./unzipped_data/tn.movie_budgets.csv",index_col=0)
print("Summary of the data on movie info from The Numbers:")
print(df_thenumbers.info())
df_thenumbers.head()

Summary of the data on movie info from The Numbers:
<class 'pandas.core.frame.DataFrame'>
Index: 5782 entries, 1 to 82
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   object
 3   domestic_gross     5782 non-null   object
 4   worldwide_gross    5782 non-null   object
dtypes: object(5)
memory usage: 271.0+ KB
None


,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


This table will be the main source we will use to determine the ROI metric. It has more entries compared to the one from BOX Office Mojo; it has no NaNs so all the data can be used. Furthermore, it has data on the production budget, which is missing from the data extracted from the Box Office Mojo. One might consider scraping the web to find production budgets for the movies listed in the Box Office Mojo database but we choose to rely solely on this table for now.  

Note that the production_bbudget, domestic_gross and worldwide_gross columns are of the type object. In order to easily do mathematical operations on them, we need to convert them to float. 

In [652]:
cols_to_convert = ["production_budget", "domestic_gross","worldwide_gross"]

for col in cols_to_convert:
    df_thenumbers[col] = df_thenumbers[col].str.replace(",","").replace("\$","",regex=True).astype(float)

In [653]:
df_thenumbers[cols_to_convert].describe() # take a look at some statistics for the data

,production_budget,domestic_gross,worldwide_gross
count,5.782000e+03,5.782000e+03,5.782000e+03
mean,3.158776e+07,4.187333e+07,9.148746e+07
std,4.181208e+07,6.824060e+07,1.747200e+08
min,1.100000e+03,0.000000e+00,0.000000e+00
25%,5.000000e+06,1.429534e+06,4.125415e+06
50%,1.700000e+07,1.722594e+07,2.798445e+07
75%,4.000000e+07,5.234866e+07,9.764584e+07
max,4.250000e+08,9.366622e+08,2.776345e+09


## 3. Data Preparation

### 3.1. Box Office Mojo

**Can we find movie budgets using web scraping?**

In this section, we will use web scraping to find the budget for movies listed in the Box Office Mojo dataset. We will first use the movie name to join the BOM dataset with the movie_basics table from the IMDB dataset. Then using the movie_id from the IMDB dataset, we can scrape the corresponding webpages to find the estimated budgets for each movie. 

In [654]:
q = """SELECT movie_id, primary_title, genres FROM movie_basics""" # query to read the movie_basics table
df_imdb_moviebasics = pd.read_sql(q, conn) # store the table in a pandas dataframe
df_imdb_moviebasics.rename(columns = {"primary_title":"title"},inplace=True); # change primary_title column name to match the BOM dataset
df_bom_imdb_merged = pd.merge(df_bom, df_imdb_moviebasics,how="inner", on="title") # merge the two tables
print("percentage of missing data:",(len(df_bom)-len(df_bom_imdb_merged))/len(df_bom)*100)
df_bom_imdb_merged.head(10)

percentage of missing data: 0.6200177147918512


,title,studio,domestic_gross,foreign_gross,year,movie_id,genres
0,Toy Story 3,BV,415000000.0,652000000.0,2010,tt0435761,"Adventure,Animation,Comedy"
1,Inception,WB,292600000.0,535700000.0,2010,tt1375666,"Action,Adventure,Sci-Fi"
2,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,tt0892791,"Adventure,Animation,Comedy"
3,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000.0,2010,tt1325004,"Adventure,Drama,Fantasy"
4,Iron Man 2,Par.,312400000.0,311500000.0,2010,tt1228705,"Action,Adventure,Sci-Fi"
5,Tangled,BV,200800000.0,391000000.0,2010,tt0398286,"Adventure,Animation,Comedy"
6,Despicable Me,Uni.,251500000.0,291600000.0,2010,tt1323594,"Animation,Comedy,Family"
7,How to Train Your Dragon,P/DW,217600000.0,277300000.0,2010,tt0892769,"Action,Adventure,Animation"
8,The Chronicles of Narnia: The Voyage of the Da...,Fox,104400000.0,311300000.0,2010,tt0980970,"Adventure,Family,Fantasy"
9,The King's Speech,Wein.,135500000.0,275400000.0,2010,tt1504320,"Biography,Drama,History"


Let's remove portions of the data that have missing information on the foreign and domestic revenue. As mentioned in Section 2.1, this is about 40% of the data. 

In [655]:
df_bom_imdb_merged.dropna(subset=["domestic_gross", "foreign_gross"], inplace=True) 

**Note:** IMDB prevents scraping of its webpages in this manner. We may need to scrape other sources such as OMDB to get this data. This may be a lot of additional effort. Let's first see how much additional data does this really provide us with?  

**Question:** How much additional data on movies does the Box Office Mojo dataset contain compared to the dataset from The Numbers? 

In [656]:
df_thenumbers.rename(columns={"movie":"title"},inplace=True)
df_thenumbers["title"] = df_thenumbers["title"].str.title()
print("percentage of additional data contained in the Box Office Mojo Dataset compared to the dataset from The Numbers:")
(len(df_bom_imdb_merged.index)-len(pd.merge(df_thenumbers,df_bom_imdb_merged,how="inner",on="title").index))/len(df_thenumbers.index)*100

percentage of additional data contained in the Box Office Mojo Dataset compared to the dataset from The Numbers:


15.150466966447595

The additional data is about 10%. At this stage, we make a judgement call to ignore the dataset from Box Office Mojo and solely rely on the dataset from The Numbers. 

## IMDB

In [657]:
q2 = """SELECT mvb.movie_id AS imdb_movie_id, mvb.start_year AS release_year, mvb.primary_title AS title, mvb.genres, mvr.averagerating AS imdb_average_rating
            FROM movie_basics mvb
                JOIN movie_ratings as mvr
                USING(movie_id)
"""
df_imdb_mvr = pd.read_sql(q2,conn)
df_imdb_mvr.head()
df_imdb_mvr["title"] = df_imdb_mvr["title"].str.title()
print(df_imdb_mvr.isna().sum())
df_imdb_mvr.head()

imdb_movie_id            0
release_year             0
title                    0
genres                 804
imdb_average_rating      0
dtype: int64


,imdb_movie_id,release_year,title,genres,imdb_average_rating
0,tt0063540,2013,Sunghursh,"Action,Crime,Drama",7.0
1,tt0066787,2019,One Day Before The Rainy Season,"Biography,Drama",7.2
2,tt0069049,2018,The Other Side Of The Wind,Drama,6.9
3,tt0069204,2018,Sabse Bada Sukh,"Comedy,Drama",6.1
4,tt0100275,2017,The Wandering Soap Opera,"Comedy,Drama,Fantasy",6.5


In [658]:
df_imdb_duplicate_title_year = pd.concat(g for _, g in df_imdb_mvr.groupby(["title","release_year"]) if len(g) > 1)

In [659]:
df_imdb_mvr = df_imdb_mvr[~ df_imdb_mvr.duplicated(subset=['title','release_year'])]

### 3.1. The Movie DataBase

**Replace genre ids with categories**

In [660]:
df_genre_tmdb["id"].to_list()
df_genre_tmdb["genre"].str.strip().to_list()
# make a dictionary where keys are genre ids and values are the actual genres
genre_tmdb_dict = {key:value for key,value in zip(df_genre_tmdb["id"].to_list(),df_genre_tmdb["genre"].str.strip().to_list())}

In [661]:
def replace_id_with_genre(key_list: list, genre_dict: dict) -> str:
    """Given a list of keys from a dictionary, this function replaces the keys with associated values.
    
    Args:
        key_list: list containing certain keys in a dictionary
        genre_dict: the dictionary itself
    
    Returns:
        A string containing corresponding values in genre_dict
    """
    value_list = [genre_dict.get(id,id) for id in ast.literal_eval(key_list)] 
    return ",".join(value_list)

# apply the function to the series
df_movie_tmdb["genre_ids"] = df_movie_tmdb["genre_ids"].apply(func = replace_id_with_genre, genre_dict = genre_tmdb_dict)
df_movie_tmdb.rename(columns={"genre_ids":"genres_category","vote_average":"tmdb_average_rating"},inplace=True)

In [662]:
df_movie_tmdb.head(3) # take a peek at the data

,genres_category,id,original_language,original_title,popularity,release_date,title,tmdb_average_rating,vote_count
0,"Adventure,Fantasy,Family",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"Fantasy,Adventure,Animation,Family",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"Adventure,Action,Science Fiction",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368


**Make necessary changes to data types**

In [663]:
# change the data type for release_date to datetime
df_movie_tmdb["release_date"] = pd.to_datetime(df_movie_tmdb["release_date"])
# create a column release_year that contains only the year. This will come in handy later on when doing merges
df_movie_tmdb["release_year"] = df_movie_tmdb["release_date"].dt.year

**Address entries with duplicate titles**

In [664]:
#df_movie_tmdb[df_movie_tmdb.duplicated(subset = ["title","release_year"])]
len(pd.concat(g for _, g in df_movie_tmdb.groupby(["title","id"]) if len(g) > 1))
df_movie_tmdb.drop_duplicates(subset=['title','id'], keep='first', inplace=True)

### 3.3. The Numbers

**Calculate the domestic and worldwide ROI**

Let's add another column to this table that contains the ROI. We can consider an ROI that only considers the domestic revenue and another one that considers both domestic and foregin revenues. 

In [665]:
df_thenumbers["ROI_domestic"] = (df_thenumbers["domestic_gross"] - df_thenumbers["production_budget"])/df_thenumbers["production_budget"]*100
df_thenumbers["ROI_worldwide"] = (df_thenumbers["worldwide_gross"] - df_thenumbers["production_budget"])/df_thenumbers["production_budget"]*100

In [666]:
# change the data type for release_date to datetime
df_thenumbers["release_date"] = pd.to_datetime(df_thenumbers["release_date"])
# create a column release_year that contains only the year. This will come in handy later on when doing merges
df_thenumbers["release_year"] = df_thenumbers["release_date"].dt.year

Let's sort the dataframe based on ROI from highest to lowest to identify the most profitable movies. 

In [667]:
df_thenumbers.sort_values(by="ROI_worldwide", ascending=False).head(5)

,release_date,title,production_budget,domestic_gross,worldwide_gross,ROI_domestic,ROI_worldwide,release_year
id,,,,,,,,
46,1972-06-30,Deep Throat,25000.0,45000000.0,45000000.0,179900.000000,179900.000000,1972
14,1980-03-21,Mad Max,200000.0,8750000.0,99750000.0,4275.000000,49775.000000,1980
93,2009-09-25,Paranormal Activity,450000.0,107918810.0,194183034.0,23881.957778,43051.785333,2009
80,2015-07-10,The Gallows,100000.0,22764410.0,41656474.0,22664.410000,41556.474000,2015
7,1999-07-14,The Blair Witch Project,600000.0,140539099.0,248300000.0,23323.183167,41283.333333,1999


**Add Genre and movie rating information to the dataset**

**Addressing duplicate entries**

**Question:** Are there any movies with duplicate title and year in the dataset? 

In [668]:
df_thenumbers[df_thenumbers.duplicated(subset = ["title","release_year"])]
pd.concat(g for _, g in df_thenumbers.groupby(["title","release_year"]) if len(g) > 1)

,release_date,title,production_budget,domestic_gross,worldwide_gross,ROI_domestic,ROI_worldwide,release_year
id,,,,,,,,
56,2009-06-05,Home,12000000.0,0.0,0.0,-100.0000,-100.0000,2009
60,2009-04-23,Home,500000.0,15433.0,44793168.0,-96.9134,8858.6336,2009


The data in row 56 seems suspicious. We will choose to drop this row. 

In [669]:
df_thenumbers.drop(index=56,inplace=True)

**Question:** Are there any movies with duplicate tilte? 

In [670]:
print("There are ", df_thenumbers.duplicated(subset="title").sum(), "entries with duplicate titles.")
pd.concat(g for _, g in df_thenumbers.groupby(["title"]) if len(g) > 1).head(6)

There are  82 entries with duplicate titles.


,release_date,title,production_budget,domestic_gross,worldwide_gross,ROI_domestic,ROI_worldwide,release_year
id,,,,,,,,
71,1954-12-23,"20,000 Leagues Under The Sea",5000000.0,28200000.0,28200000.0,464.000000,464.000000,1954
15,1916-12-24,"20,000 Leagues Under The Sea",200000.0,8000000.0,8000000.0,3900.000000,3900.000000,1916
49,2010-04-30,A Nightmare On Elm Street,35000000.0,63075011.0,117729621.0,80.214317,236.370346,2010
17,1984-11-09,A Nightmare On Elm Street,1800000.0,25504513.0,25504513.0,1316.917389,1316.917389,1984
81,2019-05-24,Aladdin,182000000.0,246734314.0,619234314.0,35.568304,240.238634,2019
33,1992-11-11,Aladdin,28000000.0,217350219.0,504050219.0,676.250782,1700.179354,1992


There are clearly movies with duplicate titles but different release years. If we merge this dataset with the ones fro IMDB/TMDB using both title and release year, we will get a lot of NaNs because slight differences in the title string or release year will prevent a match. I prefer to join only based on title but the duplicate titles in both this dataset and the one from IMDB will cause issues because upon merge, they will get duplicated many times, creating entries with fake data. We don't want that. Since movies with duplicate titles are only a small portion of the whole database, we will store them in a separate dataframe for now and will drop them from the main dataset. We can work on integrating these entries in a neater way later on. 

In [671]:
df_tn_duplicate_titles = pd.concat(g for _, g in df_thenumbers.groupby(["title"]) if len(g) > 1)
df_thenumbers_noduplicates = df_thenumbers[~ df_thenumbers.duplicated(subset="title")]

In [709]:
df_imdb_mvr.imdb_movie_id.info()

<class 'pandas.core.series.Series'>
RangeIndex: 73264 entries, 0 to 73263
Series name: imdb_movie_id
Non-Null Count  Dtype 
--------------  ----- 
73264 non-null  object
dtypes: object(1)
memory usage: 572.5+ KB


In [672]:
# merge the dataset from The Numbers with the dataset from IMDB to record the genres as well as ratings
df_tn_imdb_merged = pd.merge(df_thenumbers_noduplicates, df_imdb_mvr, on=['title','release_year'], how="left")
# print(len(df_thenumbers_noduplicates.index))
# print(len(df_tn_imdb_merged.index))
# print(df_tn_imdb_merged.isna().sum())

df_merged_all = pd.merge(df_tn_imdb_merged, df_movie_tmdb[["title","genres_category","tmdb_average_rating"]], on="title", how="left")

df_merged_all["genres"]=df_merged_all["genres"].fillna(df_merged_all["genres_category"])

df_merged_all.drop(columns = "genres_category", inplace=True)
print(df_merged_all.isna().sum())

df_merged_all[(df_merged_all['genres'].isna()) & (df_merged_all['release_year']>=2010)]['release_year'].value_counts().sort_values(ascending=False)

release_date              0
title                     0
production_budget         0
domestic_gross            0
worldwide_gross           0
ROI_domestic              0
ROI_worldwide             0
release_year              0
imdb_movie_id          4237
genres                 3718
imdb_average_rating    4237
tmdb_average_rating    4117
dtype: int64


release_year
2015    85
2010    80
2011    47
2016    43
2012    39
2014    38
2013    36
2017    31
2019    21
2018    14
2020     3
Name: count, dtype: int64

In [673]:
df_merged_all.sort_values(by="ROI_worldwide", ascending=False).head(5)

,release_date,title,production_budget,domestic_gross,worldwide_gross,ROI_domestic,ROI_worldwide,release_year,imdb_movie_id,genres,imdb_average_rating,tmdb_average_rating
5790,1972-06-30,Deep Throat,25000.0,45000000.0,45000000.0,179900.000000,179900.000000,1972,NaN,NaN,NaN,NaN
5666,1980-03-21,Mad Max,200000.0,8750000.0,99750000.0,4275.000000,49775.000000,1980,NaN,NaN,NaN,NaN
5541,2009-09-25,Paranormal Activity,450000.0,107918810.0,194183034.0,23881.957778,43051.785333,2009,NaN,NaN,NaN,NaN
5724,2015-07-10,The Gallows,100000.0,22764410.0,41656474.0,22664.410000,41556.474000,2015,tt2309260,"Horror,Mystery,Thriller",4.2,4.8
5456,1999-07-14,The Blair Witch Project,600000.0,140539099.0,248300000.0,23323.183167,41283.333333,1999,NaN,NaN,NaN,NaN


In [740]:
df_nan_2010_onwards = df_merged_all[(df_merged_all['genres'].isna()) & (df_merged_all['release_year']>=2010)]
idx=[]
idx_df_nan=[]
# df_imdb_mvr.reset_index(inplace=True) # uncomment if running from scratch

for i in range(0,len(df_nan_2010_onwards.index)):
        if (df_nan_2010_onwards["title"].iloc[i] in df_imdb_mvr["title"].to_list()):
            idx_df_nan.append(i)
            idx.append(df_imdb_mvr["title"].index[df_imdb_mvr["title"].str.contains(df_nan_2010_onwards["title"].iloc[i])].to_list())
            
            #print(idx[j])
            # print([df_imdb_mvr["title"].iloc[idx_el] for idx_el in idx[-1]])
            idx_closest_year = (df_imdb_mvr["release_year"].iloc[idx[-1]]-df_nan_2010_onwards["release_year"].iloc[i]).idxmin()
            if numpy.abs((df_imdb_mvr["release_year"].iloc[idx[-1]]-df_nan_2010_onwards["release_year"].iloc[i]).min())<=2:                
                columns_to_fill = ["imdb_movie_id", "genres", "imdb_average_rating"]
                for col in columns_to_fill:
                    # print(df_nan_2010_onwards[["title","release_year"]].iloc[i])
                    # print(df_imdb_mvr[["title","release_year"]].iloc[idx_closest_year])
                    df_nan_2010_onwards.iloc[i,df_nan_2010_onwards.columns.get_loc(col)] = df_imdb_mvr.iloc[idx_closest_year,df_imdb_mvr.columns.get_loc(col)] 
                                            
print(df_nan_2010_onwards.isna().sum()/len(df_nan_2010_onwards.index))

release_date           0.000000
title                  0.000000
production_budget      0.000000
domestic_gross         0.000000
worldwide_gross        0.000000
ROI_domestic           0.000000
ROI_worldwide          0.000000
release_year           0.000000
imdb_movie_id          0.773455
genres                 0.773455
imdb_average_rating    0.773455
tmdb_average_rating    1.000000
dtype: float64


In [767]:
df_nan_2010_onwards.rename(columns={"genres":"filled_genres", "imdb_movie_id":"filled_imdb_movie_id", "imdb_average_rating":"filled_imdb_average_rating"}, inplace=True)
df_new = pd.merge(df_merged_all, df_nan_2010_onwards[["title","filled_genres", "filled_imdb_movie_id", "filled_imdb_average_rating"]], on="title", how = "left")

print(df_new.isna().sum())
na_cols_to_fill = ["imdb_movie_id","genres","imdb_average_rating"]
cols_to_fill_with = ["filled_imdb_movie_id","filled_genres","filled_imdb_average_rating"]

for na_col, col_to_fill in zip(na_cols_to_fill, cols_to_fill_with):
    df_new[na_col] = df_new[na_col].fillna(df_new[col_to_fill])

df_new.drop(columns = cols_to_fill_with, inplace=True)
df_new.isna().sum()

release_date                     0
title                            0
production_budget                0
domestic_gross                   0
worldwide_gross                  0
ROI_domestic                     0
ROI_worldwide                    0
release_year                     0
imdb_movie_id                 4237
genres                        3718
imdb_average_rating           4237
tmdb_average_rating           4117
filled_genres                 5727
filled_imdb_movie_id          5727
filled_imdb_average_rating    5727
dtype: int64


C:\Users\setar\AppData\Local\Temp\ipykernel_4076\1133414722.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nan_2010_onwards.rename(columns={"genres":"filled_genres", "imdb_movie_id":"filled_imdb_movie_id", "imdb_average_rating":"filled_imdb_average_rating"}, inplace=True)


release_date              0
title                     0
production_budget         0
domestic_gross            0
worldwide_gross           0
ROI_domestic              0
ROI_worldwide             0
release_year              0
imdb_movie_id          4138
genres                 3619
imdb_average_rating    4138
tmdb_average_rating    4117
dtype: int64

Some brainstorming: 

- address: RT table just has ids but no movie title. Maybe some web scraping could help? Scraping won't help with this... We will consider the data from RT separately to make conclusions. It cannot be mixed with data from other data bases. 
- ~~include the genre of the movie in the same table from The Numbers.~~
- ~~Identify the most profitable movies then look through the rest of the tables to identify the genres/ratings, directors.~~ 
- do a group by based on attributes then do averages and sort to find the most profitable genres. 
- write functions that visualize the ROI based on attributes. (e.g. box plot for ROI versus genre. Includes average and quartiles naturally).
- ~~a neat idea: movie name from Box Office Mojo --> IMDB id --> web scraping to get the budget then add to the table from The Numbers.~~
- possible challenge: single movie multiple genres listed... for genre analysis, can we do a regular groupby? Probably not... how to deal with it?